In [ ]:
import numpy as np
from coverage import *

from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import branca
import folium


In [ ]:
num_days = 1
tles = gen_sats(sat_nos=[39084,49260])

# Generate the instrument from basic specs
# times = gen_times(start_yr=2022, start_mo=6, start_day=10, days=num_days, step_min=1) # Adjusted TIRS for 1 min FOVs
# inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=4000, mm=True)

times = gen_times(start_yr=2022, start_mo=6, start_day=15, days=0.25, step_min=0.22) # Actual TIRS
# inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=1800, mm=True)

In [ ]:
cmap = {'LANDSAT 9': '#0000ffff', 'LANDSAT 8': '#ff0000ff'}

In [ ]:
ssps = []
for tle in tles:
    sat = tle[0]
    for time in times:
        ssp_lat, ssp_lon, d = get_los(sat, time)

        xyz_dist_rates = sat.at(time).frame_xyz_and_velocity(itrs)
        # xyz_dist = xyz_dist_rates[0]
        z_rate = xyz_dist_rates[1]

        # Descending only filter:
        if z_rate.km_per_s[2] < 0:
            orbit = "descending"
        if z_rate.km_per_s[2] > 0:
            orbit = "ascending"

        ssps.append(
            {
            'time' : time.utc_datetime(),
            'geometry' : Point(ssp_lon.degrees, ssp_lat.degrees),
            'range' : d,
            'color' : cmap[sat.name],
            'orbit' : orbit
            }
        )

In [ ]:
ssp_df = gpd.GeoDataFrame(ssps, crs="epsg:4326")
ssp_df.drop("time", axis=1).explore(
    color="color", tooltip=True, style_kwds={"stroke":False}
)

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.explore()

In [ ]:
land_ssp_df = gpd.sjoin(ssp_df, world, predicate='within', how='inner')

land_ssp_df[land_ssp_df.orbit == "descending"].drop(
    'time', axis=1
    ).explore(
        color="color", tooltip=False, style_kwds={"stroke":False}
        )

In [ ]:
sat.model.no_kozai 

In [ ]:
from sgp4.api import Satrec, WGS72

satrec = Satrec()
satrec.sgp4init(
    WGS72,           # gravity model
    'i',             # 'a' = old AFSPC mode, 'i' = improved mode
    5,               # satnum: Satellite number
    18441.785,       # epoch: days since 1949 December 31 00:00 UT
    2.8098e-05,      # bstar: drag coefficient (/earth radii)
    6.969196665e-13, # ndot: ballistic coefficient (revs/day)
    0.0,             # nddot: second derivative of mean motion (revs/day^3)
    0.1859667,       # ecco: eccentricity
    5.7904160274885, # argpo: argument of perigee (radians)
    0.5980929187319, # inclo: inclination (radians)
    0.3373093125574, # mo: mean anomaly (radians)
    0.0472294454407, # no_kozai: mean motion (radians/minute)
    6.0863854713832, # nodeo: right ascension of ascending node (radians)
)

In [ ]:
# model.satnum
#     The unique satellite NORAD catalog number given in the TLE file.
# model.classification
#     Satellite classification, or else 'U' for “Unknown”
# model.intldesg
#     International designator
# model.epochyr
#     Full four-digit year of this element set’s epoch moment.
# model.epochdays
#     Fractional days into the year of the epoch moment.
# model.jdsatepoch
#     Julian date of the epoch (computed from epochyr and epochdays).
# model.ndot
#     First time derivative of the mean motion (ignored by SGP4).
# model.nddot
#     Second time derivative of the mean motion (ignored by SGP4).
# model.bstar
#     Ballistic drag coefficient B* in inverse earth radii.
# model.ephtype
#     Ephemeris type (ignored by SGP4 as determination now automatic)
# model.elnum
#     Element number
# model.inclo
#     Inclination in radians.
# model.nodeo
#     Right ascension of ascending node in radians.
# model.ecco
#     Eccentricity.
# model.argpo
#     Argument of perigee in radians.
# model.mo
#     Mean anomaly in radians.
# model.no_kozai
#     Mean motion in radians per minute.
# model.revnum
#     Revolution number at epoch [Revs]